# 文本相似度检测

被检测的文本数据集，分为了三类

In [1]:
# 定义文本对
texts = [
    # 完全相同的文本对
    ('今年夏天格外炎热，许多城市的温度都达到了历史新高，专家建议民众减少午后外出，多补充水分，尽量待在空调房内避暑。',
     '今年夏天格外炎热，许多城市的温度都达到了历史新高，专家建议民众减少午后外出，多补充水分，尽量待在空调房内避暑。'),
    # 轻微修改的文本对
    ('随着科技的迅速发展，人工智能在医疗、教育、交通等多个领域的应用越来越广泛，它正逐渐改变着人们的生活方式和工作模式。',
     '人工智能的快速发展推动了其在医疗、教育、交通等众多领域的广泛应用，这正在慢慢地改变人们的日常生活和工作方式。'),
    # 概念上相似但表达方式不同的文本对
    ('今年入冬以来，北方多地出现了严重的雾霾天气，能见度低，对人们的健康和日常出行造成了极大影响。',
     '自从这个冬季开始，北方地区频繁遭受雾霾侵袭，低能见度严重威胁到居民健康及其日常生活。'),
    ('太阳从东边升起', '太阳从西边升起')
]

### 余弦相似度cosine_similarity

In [2]:
# -*- coding: utf-8 -*-

# 正则包
import re
# html 包
import html
# 自然语言处理包
import jieba
import jieba.analyse
# 机器学习包
from sklearn.metrics.pairwise import cosine_similarity


class CosineSimilarity(object):
    """
    余弦相似度
    """

    def __init__(self, content_x1, content_y2):
        self.s1 = content_x1
        self.s2 = content_y2

    @staticmethod
    def extract_keyword(content):  # 提取关键词
        # 切割
        seg = [i for i in jieba.cut(content, cut_all=True) if i != '']
        # 提取关键词
        keywords = jieba.analyse.extract_tags("|".join(seg), topK=200, withWeight=False)
        print(keywords)
        return keywords

    @staticmethod
    def one_hot(word_dict, keywords):  # oneHot编码
        # cut_code = [word_dict[word] for word in keywords]
        cut_code = [0] * len(word_dict)
        for word in keywords:
            cut_code[word_dict[word]] += 1
        return cut_code

    def main(self):
        # 去除停用词
        jieba.analyse.set_stop_words('data/stopwords.txt')

        # 提取关键词
        keywords1 = self.extract_keyword(self.s1)
        keywords2 = self.extract_keyword(self.s2)
        # 词的并集
        union = set(keywords1).union(set(keywords2))
        # 编码
        word_dict = {}
        i = 0
        for word in union:
            word_dict[word] = i
            i += 1
        # oneHot编码
        s1_cut_code = self.one_hot(word_dict, keywords1)
        s2_cut_code = self.one_hot(word_dict, keywords2)
        # 余弦相似度计算
        sample = [s1_cut_code, s2_cut_code]
        # 除零处理
        try:
            sim = cosine_similarity(sample)
            return sim[1][0]
        except Exception as e:
            print(e)
            return 0.0


# 计算并打印相似度
for text_pair in texts:
    cs = CosineSimilarity(*text_pair)
    similarity = cs.main()
    print(f"文本1: \"{text_pair[0]}\"")
    print(f"文本2: \"{text_pair[1]}\"")
    print(f"相似度: {similarity}\n")

D:\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
D:\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
D:\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
D:\Anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Edgar\AppData\Local\Temp\jieba.cache
Loading model cost 0.742 seconds.
Prefix dict has been built successfully.


['充水', '天格', '今年夏天', '避暑', '专家建议', '炎热', '外出', '空调', '水分', '午后', '民众', '夏天', '温度', '新高', '补充', '专家', '建议', '减少', '城市', '历史']
['充水', '天格', '今年夏天', '避暑', '专家建议', '炎热', '外出', '空调', '水分', '午后', '民众', '夏天', '温度', '新高', '补充', '专家', '建议', '减少', '城市', '历史']
文本1: "今年夏天格外炎热，许多城市的温度都达到了历史新高，专家建议民众减少午后外出，多补充水分，尽量待在空调房内避暑。"
文本2: "今年夏天格外炎热，许多城市的温度都达到了历史新高，专家建议民众减少午后外出，多补充水分，尽量待在空调房内避暑。"
相似度: 1.0000000000000002

['人工智能', '智能', '人工', '医疗', '多个', '模式', '交通', '科技', '教育', '越来越', '领域', '改变', '方式', '生活', '工作', '发展']
['人工智能', '日常生活', '广泛应用', '智能', '日常', '人工', '医疗', '慢慢', '交通', '众多', '快速', '教育', '领域', '推动', '改变', '方式', '生活', '工作', '发展']
文本1: "随着科技的迅速发展，人工智能在医疗、教育、交通等多个领域的应用越来越广泛，它正逐渐改变着人们的生活方式和工作模式。"
文本2: "人工智能的快速发展推动了其在医疗、教育、交通等众多领域的广泛应用，这正在慢慢地改变人们的日常生活和工作方式。"
相似度: 0.6882472016116852

['入冬', '能见度', '出行', '日常', '天气', '北方', '健康', '影响']
['低能', '北方地区', '严重威胁', '能见度', '侵袭', '日常生活', '日常', '遭受', '频繁', '冬季', '威胁', '北方', '健康', '居民', '生活', '地区']
文本1: "今年入冬以来，北方多地出现了严重的雾霾天气，能见度低，对人们的健康和日常出行造成了极大影响。"
文本2: "自从这个冬季开始，北方地

### jaccard相似度

In [3]:
# -*- coding: utf-8 -*-

# 正则包
import re
# 自然语言处理包
import jieba
import jieba.analyse
# html 包
import html


class JaccardSimilarity(object):
    """
    jaccard相似度
    """

    def __init__(self, content_x1, content_y2):
        self.s1 = content_x1
        self.s2 = content_y2

    @staticmethod
    def extract_keyword(content):  # 提取关键词
        # 切割
        seg = [i for i in jieba.cut(content, cut_all=True) if i != '']
        # 提取关键词
        keywords = jieba.analyse.extract_tags("|".join(seg), topK=200, withWeight=False)
        print(keywords)
        return keywords

    def main(self):
        # 去除停用词
        jieba.analyse.set_stop_words('data/stopwords.txt')

        # 分词与关键词提取
        keywords_x = self.extract_keyword(self.s1)
        keywords_y = self.extract_keyword(self.s2)

        # jaccard相似度计算
        intersection = len(list(set(keywords_x).intersection(set(keywords_y))))
        union = len(list(set(keywords_x).union(set(keywords_y))))
        # 除零处理
        sim = float(intersection) / union if union != 0 else 0
        return sim


# 计算并打印相似度
for text_pair in texts:
    js = JaccardSimilarity(*text_pair)
    similarity = js.main()
    print(f"文本1: \"{text_pair[0]}\"")
    print(f"文本2: \"{text_pair[1]}\"")
    print(f"Jaccard相似度: {similarity}\n")

['充水', '天格', '今年夏天', '避暑', '专家建议', '炎热', '外出', '空调', '水分', '午后', '民众', '夏天', '温度', '新高', '补充', '专家', '建议', '减少', '城市', '历史']
['充水', '天格', '今年夏天', '避暑', '专家建议', '炎热', '外出', '空调', '水分', '午后', '民众', '夏天', '温度', '新高', '补充', '专家', '建议', '减少', '城市', '历史']
文本1: "今年夏天格外炎热，许多城市的温度都达到了历史新高，专家建议民众减少午后外出，多补充水分，尽量待在空调房内避暑。"
文本2: "今年夏天格外炎热，许多城市的温度都达到了历史新高，专家建议民众减少午后外出，多补充水分，尽量待在空调房内避暑。"
Jaccard相似度: 1.0

['人工智能', '智能', '人工', '医疗', '多个', '模式', '交通', '科技', '教育', '越来越', '领域', '改变', '方式', '生活', '工作', '发展']
['人工智能', '日常生活', '广泛应用', '智能', '日常', '人工', '医疗', '慢慢', '交通', '众多', '快速', '教育', '领域', '推动', '改变', '方式', '生活', '工作', '发展']
文本1: "随着科技的迅速发展，人工智能在医疗、教育、交通等多个领域的应用越来越广泛，它正逐渐改变着人们的生活方式和工作模式。"
文本2: "人工智能的快速发展推动了其在医疗、教育、交通等众多领域的广泛应用，这正在慢慢地改变人们的日常生活和工作方式。"
Jaccard相似度: 0.5217391304347826

['入冬', '能见度', '出行', '日常', '天气', '北方', '健康', '影响']
['低能', '北方地区', '严重威胁', '能见度', '侵袭', '日常生活', '日常', '遭受', '频繁', '冬季', '威胁', '北方', '健康', '居民', '生活', '地区']
文本1: "今年入冬以来，北方多地出现了严重的雾霾天气，能见度低，对人们的健康和日常出行造成了极大影响。"
文本2: "自从这个冬季开始，北方地区

### 编辑距离Levenshtein

In [4]:
# -*- coding: utf-8 -*-

# 正则包
import re
# html 包
import html
# 自然语言处理包
import jieba
import jieba.analyse
# 编辑距离包
import Levenshtein


class LevenshteinSimilarity(object):
    """
    编辑距离
    """

    def __init__(self, content_x1, content_y2):
        self.s1 = content_x1
        self.s2 = content_y2

    @staticmethod
    def extract_keyword(content):  # 提取关键词
        # 切割
        seg = [i for i in jieba.cut(content, cut_all=True) if i != '']
        # 提取关键词
        keywords = jieba.analyse.extract_tags("|".join(seg), topK=200, withWeight=False)
        return keywords

    def main(self):
        # 去除停用词
        jieba.analyse.set_stop_words('data/stopwords.txt')

        # 提取关键词
        keywords1 = ', '.join(self.extract_keyword(self.s1))
        keywords2 = ', '.join(self.extract_keyword(self.s2))

        # ratio计算2个字符串的相似度，它是基于最小编辑距离
        distances = Levenshtein.ratio(keywords1, keywords2)
        return distances


# 计算并打印相似度
for text_pair in texts:
    ls = LevenshteinSimilarity(*text_pair)
    similarity = ls.main()
    print(f"文本1: \"{text_pair[0]}\"")
    print(f"文本2: \"{text_pair[1]}\"")
    print(f"Levenshtein相似度: {similarity}\n")

文本1: "今年夏天格外炎热，许多城市的温度都达到了历史新高，专家建议民众减少午后外出，多补充水分，尽量待在空调房内避暑。"
文本2: "今年夏天格外炎热，许多城市的温度都达到了历史新高，专家建议民众减少午后外出，多补充水分，尽量待在空调房内避暑。"
Levenshtein相似度: 1.0

文本1: "随着科技的迅速发展，人工智能在医疗、教育、交通等多个领域的应用越来越广泛，它正逐渐改变着人们的生活方式和工作模式。"
文本2: "人工智能的快速发展推动了其在医疗、教育、交通等众多领域的广泛应用，这正在慢慢地改变人们的日常生活和工作方式。"
Levenshtein相似度: 0.7172413793103448

文本1: "今年入冬以来，北方多地出现了严重的雾霾天气，能见度低，对人们的健康和日常出行造成了极大影响。"
文本2: "自从这个冬季开始，北方地区频繁遭受雾霾侵袭，低能见度严重威胁到居民健康及其日常生活。"
Levenshtein相似度: 0.45999999999999996

文本1: "太阳从东边升起"
文本2: "太阳从西边升起"
Levenshtein相似度: 0.9


### MinHash

In [5]:
# -*- coding: utf-8 -*-

# 正则包
import re
# 自然语言处理包
import jieba
import jieba.analyse
# html 包
import html
# 数据集处理包
from datasketch import MinHash


class MinHashSimilarity(object):
    """
    MinHash
    """

    def __init__(self, content_x1, content_y2):
        self.s1 = content_x1
        self.s2 = content_y2

    @staticmethod
    def extract_keyword(content):  # 提取关键词
        # 切割
        seg = [i for i in jieba.cut(content, cut_all=True) if i != '']
        # 提取关键词
        keywords = jieba.analyse.extract_tags("|".join(seg), topK=200, withWeight=False)
        return keywords

    def main(self):
        # 去除停用词
        jieba.analyse.set_stop_words('data/stopwords.txt')

        # MinHash计算
        m1, m2 = MinHash(), MinHash()
        # 提取关键词
        s1 = self.extract_keyword(self.s1)
        s2 = self.extract_keyword(self.s2)

        for data in s1:
            m1.update(data.encode('utf8'))
        for data in s2:
            m2.update(data.encode('utf8'))

        return m1.jaccard(m2)


# 计算并打印相似度
for text_pair in texts:
    mhs = MinHashSimilarity(*text_pair)
    similarity = mhs.main()
    print(f"文本1: \"{text_pair[0]}\"")
    print(f"文本2: \"{text_pair[1]}\"")
    print(f"MinHash相似度: {similarity}\n")

文本1: "今年夏天格外炎热，许多城市的温度都达到了历史新高，专家建议民众减少午后外出，多补充水分，尽量待在空调房内避暑。"
文本2: "今年夏天格外炎热，许多城市的温度都达到了历史新高，专家建议民众减少午后外出，多补充水分，尽量待在空调房内避暑。"
MinHash相似度: 1.0

文本1: "随着科技的迅速发展，人工智能在医疗、教育、交通等多个领域的应用越来越广泛，它正逐渐改变着人们的生活方式和工作模式。"
文本2: "人工智能的快速发展推动了其在医疗、教育、交通等众多领域的广泛应用，这正在慢慢地改变人们的日常生活和工作方式。"
MinHash相似度: 0.578125

文本1: "今年入冬以来，北方多地出现了严重的雾霾天气，能见度低，对人们的健康和日常出行造成了极大影响。"
文本2: "自从这个冬季开始，北方地区频繁遭受雾霾侵袭，低能见度严重威胁到居民健康及其日常生活。"
MinHash相似度: 0.2265625

文本1: "太阳从东边升起"
文本2: "太阳从西边升起"
MinHash相似度: 0.53125


### SimHash + 海明距离

In [6]:
# -*- coding: utf-8 -*-

# 正则
import re
# html 包
import html
# 数学包
import math
# 自然语言处理包
import jieba
import jieba.analyse


class SimHashSimilarity(object):
    """
    SimHash
    """

    def __init__(self, content_x1, content_y2):
        self.s1 = content_x1
        self.s2 = content_y2

    @staticmethod
    def get_bin_str(source):  # 字符串转二进制
        if source == "":
            return 0
        else:
            t = ord(source[0]) << 7
            m = 1000003
            mask = 2 ** 128 - 1
            for c in source:
                t = ((t * m) ^ ord(c)) & mask
            t ^= len(source)
            if t == -1:
                t = -2
            t = bin(t).replace('0b', '').zfill(64)[-64:]
            return str(t)

    @staticmethod
    def extract_keyword(content):  # 提取关键词
        # 正则过滤 html 标签
        re_exp = re.compile(r'(<style>.*?</style>)|(<[^>]+>)', re.S)
        content = re_exp.sub(' ', content)
        # html 转义符实体化
        content = html.unescape(content)
        # 切割
        seg = [i for i in jieba.cut(content, cut_all=True) if i != '']
        # 提取关键词
        keywords = jieba.analyse.extract_tags("|".join(seg), topK=200, withWeight=True)
        return keywords

    def run(self, keywords):
        ret = []
        for keyword, weight in keywords:
            bin_str = self.get_bin_str(keyword)
            key_list = []
            for c in bin_str:
                weight = math.ceil(weight)
                if c == "1":
                    key_list.append(int(weight))
                else:
                    key_list.append(-int(weight))
            ret.append(key_list)
        # 对列表进行"降维"
        rows = len(ret)
        cols = len(ret[0])
        result = []
        for i in range(cols):
            tmp = 0
            for j in range(rows):
                tmp += int(ret[j][i])
            if tmp > 0:
                tmp = "1"
            elif tmp <= 0:
                tmp = "0"
            result.append(tmp)
        return "".join(result)

    def main(self):
        # 去除停用词
        jieba.analyse.set_stop_words('data/stopwords.txt')

        # 提取关键词
        s1 = self.extract_keyword(self.s1)
        s2 = self.extract_keyword(self.s2)

        sim_hash1 = self.run(s1)
        sim_hash2 = self.run(s2)
        # print(f'相似哈希指纹1: {sim_hash1}\n相似哈希指纹2: {sim_hash2}')
        length = 0
        for index, char in enumerate(sim_hash1):
            if char == sim_hash2[index]:
                continue
            else:
                length += 1
        return length


# 实例化SimHashSimilarity并计算相似度
for text_pair in texts:
    simhash_similarity = SimHashSimilarity(*text_pair)
    hamming_distance = simhash_similarity.main()
    similarity = 1 - hamming_distance / 64.0  # 假设SimHash值长度为64位
    print(f"文本1: \"{text_pair[0]}\"")
    print(f"文本2: \"{text_pair[1]}\"")
    print(f"汉明距离: {hamming_distance}")
    print(f"相似度: {similarity}\n")

文本1: "今年夏天格外炎热，许多城市的温度都达到了历史新高，专家建议民众减少午后外出，多补充水分，尽量待在空调房内避暑。"
文本2: "今年夏天格外炎热，许多城市的温度都达到了历史新高，专家建议民众减少午后外出，多补充水分，尽量待在空调房内避暑。"
汉明距离: 0
相似度: 1.0

文本1: "随着科技的迅速发展，人工智能在医疗、教育、交通等多个领域的应用越来越广泛，它正逐渐改变着人们的生活方式和工作模式。"
文本2: "人工智能的快速发展推动了其在医疗、教育、交通等众多领域的广泛应用，这正在慢慢地改变人们的日常生活和工作方式。"
汉明距离: 24
相似度: 0.625

文本1: "今年入冬以来，北方多地出现了严重的雾霾天气，能见度低，对人们的健康和日常出行造成了极大影响。"
文本2: "自从这个冬季开始，北方地区频繁遭受雾霾侵袭，低能见度严重威胁到居民健康及其日常生活。"
汉明距离: 20
相似度: 0.6875

文本1: "太阳从东边升起"
文本2: "太阳从西边升起"
汉明距离: 21
相似度: 0.671875


## 数据集评测各个算法的性能

In [20]:
import csv
from sklearn.metrics import accuracy_score
from model import *

# 读取CSV文件
texts = []  # 存储句子对和标签
with open('data/similarity.csv', 'r', encoding='gbk') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',')
    for row in reader:
        texts.append((row['sentence1'], row['sentence2'], int(row['label'])))


# 函数用于测试模型性能
def test_model(model_class, threshold=0.5):
    predictions = []
    actuals = []
    for s1, s2, label in texts:
        model = model_class(s1, s2)
        try:
            similarity = model.main()
        except:
            continue

        prediction = 1 if similarity >= threshold else 0
        predictions.append(prediction)
        actuals.append(label - 1)  # 转换label值为0或1

    return accuracy_score(actuals, predictions)


# 测试每个模型
model_classes = [CosineSimilarity, JaccardSimilarity, LevenshteinSimilarity, MinHashSimilarity, SimHashSimilarity]
threshold = 0.5  # 可以根据需要调整
for model_class in model_classes:
    accuracy = test_model(model_class, threshold)
    print(f"{model_class.__name__} Accuracy: {accuracy}")

CosineSimilarity Accuracy: 0.3203
JaccardSimilarity Accuracy: 0.3837
LevenshteinSimilarity Accuracy: 0.2632
MinHashSimilarity Accuracy: 0.4018
SimHashSimilarity Accuracy: 0.04114470842332613
